# Convolutional Neural Network

### Importing the libraries

In [57]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [58]:
tf.__version__
tf.config.experimental.list_physical_devices('GPU')

[]

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [59]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [60]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [61]:
"""
Convolutional neural network
Input image --> cnn --> prediction

B/W --> 2D array
Color --> 3D array

Preprocessing:
step 1:
feature detector (3x3 most common, 5x5, 7x7) (kernal)
if anything match up 1s in the kernal, make the feature map position + 1(stride)
create many feature maps for a image (different filters)
computer will decide which feature is important
(still preserve spacial relationships)

apply rectifier(ReLU), change all negative values to 0 (breaking up the linearity)

step 2:
Max Pooling
record maximum in feature map into pooled feature map (kernal = 2)
reducing size by 75%
reduce param

step 3:
Flattening
transform everything to a 1d array

same neural network steps as ann
for a neuron, 1 means very confident, 0 means useless
output will take in consider which neuron to use

Soft-max, cross-entropy loss
Prevent vague prediction (choose prediction value that's closer to actual value)
choose the less mean squared error model (or cross-entropy)
"Cross-entropy loss, or log loss, measures the performance of a classification model whose output is a probability value between 0 and 1. Cross-entropy loss increases as the predicted probability diverges from the actual label."

"""

cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [62]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [63]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [64]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [65]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [66]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [67]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [68]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [55]:
with tf.device('/device:GPU:1'):
    cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 45s 176ms/step - loss: 0.6896 - accuracy: 0.5385 - val_loss: 0.6248 - val_accuracy: 0.6355
Epoch 2/25
 30/250 [==>...........................] - ETA: 37s - loss: 0.6048 - accuracy: 0.6921

KeyboardInterrupt: 

## Part 4 - Making a single prediction

In [0]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [0]:
print(prediction)